In [ ]:
import sys, os
import time
import cv2

import pickle, bz2 

import tensorflow as tf
import numpy as np

sys.path.append(os.path.join(os.path.abspath(''), '../../utils'))

from diambraImitationLearning import *

In [ ]:
diambraIL_kwargs = {}
diambraIL_kwargs["hwc_dim"] = [256,256,7]
diambraIL_kwargs["n_actions"] = [9, 8]
diambraIL_kwargs["trajFilesList"] = ["./RLTraj_2020-09-04_00-16-24"] # P1
#diambraIL_kwargs["trajFilesList"] = ["./RLTraj_2020-09-04_00-08-44"] # P1P2
diambraIL_kwargs["totalCpus"] = 1

In [ ]:
env = make_diambra_imitationLearning_env(diambraImitationLearning, diambraIL_kwargs)

In [ ]:
observation = env.reset()[0]

In [ ]:
env.env_method("trajSummary")

In [ ]:
nChars = env.get_attr("nChars")[0]
n_actions = env.get_attr("n_actions")[0]
actBufLen = env.get_attr("actBufLen")[0]
playerId = env.get_attr("playerId")[0]

In [ ]:
shp = observation.shape

additionalPar = int(observation[0,0,shp[2]-1])
# 1P
nScalarAddPar = additionalPar - 2*nChars - actBufLen*(n_actions[0]+n_actions[1])

print("Additional Par = ", additionalPar)
print("N scalar actions = ", nScalarAddPar)

cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 10
currNumEp = 0

procIdx = 0

while currNumEp < maxNumEp:
       
    dummy_actions = [0 for i in range(diambraIL_kwargs["totalCpus"])]
    observation, reward, done, info = env.step(dummy_actions)
    
    observation = observation[procIdx]
    reward = reward[procIdx]
    done = done[procIdx]
    action = info[procIdx]["action"]
    #print("Action:", action)
    #print("Done = ", done)
    if done:
        observation = info[procIdx]["terminal_observation"]
    
    addPar = observation[:,:,shp[2]-1]    
    addPar = np.reshape(addPar, (-1))
    addPar = addPar[1:additionalPar+1]
    actions = addPar[0:additionalPar-nScalarAddPar-2*nChars]
        
    limAct = [actBufLen * n_actions[0], 
              actBufLen * n_actions[0] + actBufLen * n_actions[1]]
        
    moveActionsP1   = actions[0:limAct[0]]
    attackActionsP1 = actions[limAct[0]:limAct[1]]
    moveActionsP1   = np.reshape(moveActionsP1, (actBufLen,-1))
    attackActionsP1 = np.reshape(attackActionsP1, (actBufLen,-1))
    print("Move actions P1 =\n", moveActionsP1)
    print("Attack actions P1 =\n", attackActionsP1)    
        
    others = addPar[additionalPar-nScalarAddPar-2*nChars:]
    print("ownHealth = ", others[0])
    print("oppHealth = ", others[1])
    print("ownPosition = ", others[2])
    print("oppPosition = ", others[3])
        
    obs = np.array(observation).astype(np.float32)
    
    for idx in range(shp[2]-1):
        cv2.imshow("image"+str(idx), obs[:,:,idx])
    
    cv2.waitKey()
 
    cumulativeEpRew += reward
    
    if done:
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0

    if np.any(env.get_attr("exhausted")):
        break
        
print("All ep. rewards =", cumulativeEpRewAll)   
print("Mean cumulative reward =", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward =", np.std(cumulativeEpRewAll))       